In [9]:
from google.colab import files
uploaded = files.upload()


Saving data.zip to data.zip


In [11]:
!unzip data.zip -d data



Streaming output truncated to the last 5000 lines.
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard152.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard160.jpg  
  inflating: data/Ga

In [13]:

import os

# Check one of the likely folders
os.listdir('/content/data/garbage classification')[:10]  # show first 10 files



['Garbage classification']

In [14]:
import pandas as pd
import os

# Set image folder
images_dir = "/content/data/Garbage classification"

# Load splits
train_df = pd.read_csv('/content/data/one-indexed-files-notrash_train.txt', sep=r"\s+", names=['filename', 'label'])
val_df = pd.read_csv('/content/data/one-indexed-files-notrash_val.txt', sep=r"\s+", names=['filename', 'label'])
test_df = pd.read_csv('/content/data/one-indexed-files-notrash_test.txt', sep=r"\s+", names=['filename', 'label'])

# Prepend the full image path
train_df['filename'] = train_df['filename'].apply(lambda x: os.path.join(images_dir, x))
val_df['filename'] = val_df['filename'].apply(lambda x: os.path.join(images_dir, x))
test_df['filename'] = test_df['filename'].apply(lambda x: os.path.join(images_dir, x))

# Check
train_df.head()


,filename,label
0,/content/data/Garbage classification/cardboard...,3
1,/content/data/Garbage classification/paper472.jpg,2
2,/content/data/Garbage classification/paper522.jpg,2
3,/content/data/Garbage classification/glass189.jpg,1
4,/content/data/Garbage classification/glass325.jpg,1


In [17]:
# Create tf.data pipelines
# ------------------------------
IMG_SIZE = 160
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def df_to_dataset(df, shuffle=True, batch_size=BATCH):
    file_paths = df['filename'].values
    labels = df['label'].values
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    def load_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        return img, label

    ds = ds.map(load_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = df_to_dataset(train_df)
val_ds   = df_to_dataset(val_df, shuffle=False)
test_ds  = df_to_dataset(test_df, shuffle=False)

In [ ]:
# ------------------------------
# Full Robust Colab Workflow: Garbage Classification Edge AI
# ------------------------------

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# ------------------------------
# Paths
# ------------------------------
images_dir = "/content/data/Garbage classification"

# ------------------------------
# Step 0: List all images and labels
# ------------------------------
# Assuming images may be in subfolders by class, adjust if all in one folder
all_images = glob.glob(os.path.join(images_dir, '*/*.jpg'))  # if subfolders exist
if len(all_images) == 0:
    all_images = glob.glob(os.path.join(images_dir, '*.jpg'))  # fallback if all images in one folder

# Extract labels from folder names or from filename (adjust if needed)
labels = []
for f in all_images:
    folder = os.path.basename(os.path.dirname(f))
    if folder != os.path.basename(images_dir):
        labels.append(folder)
    else:
        # Take first part of filename as label (before number or underscore)
        labels.append(f.split('/')[-1].split('.')[0].split('_')[0])  # e.g., 'glass123.jpg' -> 'glass'

df = pd.DataFrame({'filename': all_images, 'label': labels})

# Encode labels as integers
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# ------------------------------
# Step 1: Split into train/val/test
# ------------------------------
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Train samples:", len(train_df))
print("Validation samples:", len(val_df))
print("Test samples:", len(test_df))

# ------------------------------
# Step 2: Create tf.data pipelines
# ------------------------------
IMG_SIZE = 160
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def df_to_dataset(df, shuffle=True, batch_size=BATCH):
    if len(df) == 0:
        raise ValueError("Dataset is empty! Cannot create tf.data pipeline.")

    file_paths = df['filename'].values
    labels = df['label'].values
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    def load_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        return img, label

    ds = ds.map(load_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = df_to_dataset(train_df)
val_ds   = df_to_dataset(val_df, shuffle=False)
test_ds  = df_to_dataset(test_df, shuffle=False)

# ------------------------------
# Step 3: Build MobileNetV2 model
# ------------------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    alpha=0.35
)
base_model.trainable = False

num_classes = len(df['label'].unique())

x = tf.keras.layers.Dropout(0.3)(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------------
# Step 4: Train Model
# ------------------------------
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/content/data/best_model.h5', save_best_only=True, monitor='val_accuracy'),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

# ------------------------------
# Step 5: Evaluate Model
# ------------------------------
model.load_weights('/content/data/best_model.h5')

loss, acc = model.evaluate(test_ds)
print("Test Accuracy:", acc)

# Classification report
y_true, y_pred = [], []
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print("Classification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


In [19]:
# Build Lightweight Model
# ------------------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    alpha=0.35
)
base_model.trainable = False

num_classes = len(train_df['label'].unique())

x = tf.keras.layers.Dropout(0.3)(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        432 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │         64 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        144 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80, 8) │        128 │ expanded_conv_de… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80, 8) │         32 │ expanded_conv_pr… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │        384 │ expanded_conv_pr… │
│ (Conv2D)            │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        192 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        432 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        192 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40, 8) │        384 │ block_1_depthwis

 Total params: 574,950 (2.19 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 410,208 (1.56 MB)

In [29]:
# Strip whitespace from filenames
train_df['filename'] = train_df['filename'].str.strip()
val_df['filename'] = val_df['filename'].str.strip()
test_df['filename'] = test_df['filename'].str.strip()

# covert folder file to lower case
folder_files = os.listdir(images_dir)
for f in folder_files:
    old_path = os.path.join(images_dir, f)
    new_path = os.path.join(images_dir, f.lower())
    if old_path != new_path:
        os.rename(old_path, new_path)

# convert train/val/filenames to lower case
train_df['filename'] = train_df['filename'].str.lower().apply(lambda x: os.path.join(images_dir, x))
val_df['filename'] = val_df['filename'].str.lower().apply(lambda x: os.path.join(images_dir, x))
test_df['filename'] = test_df['filename'].str.lower().apply(lambda x: os.path.join(images_dir, x))

In [31]:
# removing missing files
train_df = train_df[train_df['filename'].apply(os.path.exists)]
val_df   = val_df[val_df['filename'].apply(os.path.exists)]
test_df  = test_df[test_df['filename'].apply(os.path.exists)]


In [33]:
# Train Model
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/content/data/best_model.h5', save_best_only=True, monitor='val_accuracy'),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,  # you can increase if needed
    callbacks=callbacks
)

Epoch 1/10


NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
/content/data/Garbage classification/glass169.jpg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_8692]

In [23]:
# Evaluate Model
# ------------------------------
model.load_weights('/content/data/best_model.h5')
loss, acc = model.evaluate(test_ds)
print("Test Accuracy:", acc)

# Optional: Classification report
from sklearn.metrics import classification_report, confusion_matrix
y_true, y_pred = [], []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print("Classification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/data/best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)